In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

import json
import pickle

# Our generated code
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%load_ext autoreload
%autoreload 2
    
import libs.exploring as explore
import libs.visualising as visualize
import libs.cleansing as cleanse

In [ ]:
countries = pd.read_csv('./data/countries.csv')
countries = countries[['name', 'cca2', 'translations']]
countries['alias'] = countries['name']+ ', ' + countries['translations']
countries['name'] = countries['name'].str.split(",", n = 1, expand = True) 
countries = countries[['name', 'cca2', 'alias']]
countries['Forced']= 0
#countries['name_json']  = countries.name.apply(
countries.head(10)

In [ ]:
#Little Specifities to the data
countries.iloc[40, 2] = countries.iloc[40, 2] + ",Quebec" + ",Québec"
countries.iloc[80, 2] = countries.iloc[80, 2] + ",Scotland" + ",Ecosse" +" ,UK"
countries.iloc[235, 2] = countries.iloc[235, 2] + ",US"
countries.loc[250] = ["United Kingdom","UK", "UK", 1]
countries.loc[251] = ["United States","USA", "USA", 1]
#Correct for Namibia cca2 value of Nan
countries.iloc[161, 1] = "NA"

countries.isnull().any().any()


In [ ]:
for country in countries.name:
    setc = countries[countries['alias'].str.contains(country, case=False)]
    if(setc.shape[0]>1):
        print(country)

In [ ]:
countries[countries['name'].str.contains('united states', case=False)]

In [ ]:
country_set_cca2 = countries[countries.cca2.apply(lambda x: x.lower() == "gb")]
country_set_cca2

Problem was : same name can be in different country or countained in different country translation. Idea is to force the country to be the most inclusive one (since the most specified is surely so in the dataframe) so for exemple if typing is "oman" force country oman and not rOMANia (since the last one would have been typed as such ...). This should also applied for most translation. 

In [ ]:
#(id, exception associated)
exception = [(44, "China"),
             (47, "Congo"),
             (62, "Dominica"),
             (76, "France"),
             (81, "Georgia"),
             (85, "Guinea"),
             (96, "Guyana"),
             (105, "India"),
             (107, "Ireland"), 
             (108, "Iran"), 
             (126, "Laos"),
             (147, "Mali"), 
             (163, "Niger"), 
             (168, "Netherlands"),
             (173, "Oman"),
             (191, "Russia"), 
             (194, "Sudan"),
             (235, "United States"),
             (245, "Samoa")]
for num, elem in exception:
    countries.iloc[num, 3]=1

In [ ]:
countries.to_csv("./data/country_lookup.csv")

Now assign the country name in different steps : first check if the name to analysed called "name" matches one of the country name (in english) then just return that value. If not, Check that it does not match one of the country label (format cca2 by choice and since it's also in the json for the map so should use this one to compare with the map). Then, if no match yet, check if the "name" is CONTAINED in the alias (set of all country names in different languages and official/non-official). Problem: many returns possible. Check if one of these return has a value of 1 in his forced column, then assign that one. If none is forced or more then one is, don't assign and return the original country name. It would be good in the future to find these instances where none is forced so that we can add them to the list. 

This is done in cleansing.py